In [1]:
'''
This is the methods notebook for siwen Datasets.

Copyright (C) 2021 - Trupti Gore.

License: GPL-3. 
'''



'\nThis is the methods notebook for siwen Datasets.\n\nCopyright (C) 2021 - Trupti Gore.\n\nLicense: GPL-3. \n'

#### Preliminaries

In [1]:
# import necessary modules
from skimage import io
import matplotlib.pyplot as plt
import numpy as np
import skimage.filters as filters
import skimage.color as color
from PIL import Image
import pandas as pd
from skimage.util import img_as_float
from skimage.color import label2rgb, rgb2gray,gray2rgb
from skimage import segmentation,morphology,feature,measure,filters
from scipy import ndimage as ndi
import glob
from natsort import natsorted 
import os
import glob
import re
import random

In [2]:
# path to the local folder for the notebooks
BASE_DIR="/Users/Trupti/07-ImageAnalysis/" 
#path to the image folder

FOLDER="KTProtein"
IMAGE_DIR=BASE_DIR + "data/SiwenData/" + FOLDER + "/"
CSV_DIR = BASE_DIR + "Kinetochore-Particle-Tracker/AnalysisResults/SiwenMovie/" + FOLDER +"/" 
GRAPHS_DIR= BASE_DIR + "Kinetochore-Particle-Tracker/GraphAnalysis/Siwen/"
SAVED_IMAGE_DIR= BASE_DIR + "Kinetochore-Particle-Tracker/AnalysisResults/SiwenMovie/" + FOLDER +"/"



In [6]:
# import function file
%run functions-Method2-Siwen.ipynb
random.seed(3)

#### A List for the Images Folders

In [5]:
interphase_path = IMAGE_DIR + 'interphase' 
mitosis_path=IMAGE_DIR + 'mitosis' 
interphase=[]
mitosis=[]

for folder in natsorted(glob.glob(interphase_path + '/*')):
    interphase.append(interphase_path + '/' + folder.split('/')[-1].split('.')[0])
for folder in natsorted(glob.glob(mitosis_path+ '/*')):
    mitosis.append(mitosis_path+ '/' + folder.split('/')[-1].split('.')[0])    


In [6]:
mitosis,interphase

(['/Users/Trupti/07-ImageAnalysis/data/SiwenData/KTProtein/mitosis/Expt-1'],
 ['/Users/Trupti/07-ImageAnalysis/data/SiwenData/KTProtein/interphase/Expt-1'])

In [8]:
# For interphase
for i in range(len(interphase)):
    directory=interphase[i].split('/')[-1].split('.')[0] # to create a folder per experiment to save csvs and images
    print(directory)
    save_image_path= SAVED_IMAGE_DIR + 'interphase/'  + directory
    df_particle,df_mask=method_measure_signal(interphase[i],save_image_path)
    df=pd.merge(df_particle, df_mask,  how='outer',on=['label','area']
                                                       ,suffixes=('_particle', '_mask'))
    
    path = CSV_DIR  + 'interphase/' + directory 
    #print("dri is", directory)
    #print(path)
    try:
        os.makedirs(path)
    except FileExistsError:
        #directory already exists
        pass
    check_nan_in_df = df.isnull().values.any()
    if check_nan_in_df >0:
        df=df.dropna()
        print("NaNs removed from")# %s " %( directory))
   
    df.to_csv(path + '/'  +  'particles'+'.csv',index=False) 
    

Expt-1


/var/folders/nw/b5trlz_d7819ystdc2gmcwbw0000gn/T/ipykernel_45771/1830156981.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_mask_final=pd.concat([df_mask_final,df_mask],ignore_index=True)
/var/folders/nw/b5trlz_d7819ystdc2gmcwbw0000gn/T/ipykernel_45771/1830156981.py:65: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_particle_final=pd.concat([df_particle_final,df_particle],ignore_index=True)


NaNs removed from


<Figure size 640x480 with 0 Axes>

In [9]:
# For mitosis
for i in range(len(mitosis)):
    directory=mitosis[i].split('/')[-1].split('.')[0] # to create a folder per experiment to save csvs and images
    
    save_image_path= SAVED_IMAGE_DIR + 'mitosis/'  + directory
    df_particle,df_mask=method_measure_signal(mitosis[i],save_image_path)
    df=pd.merge(df_particle, df_mask,  how='outer',on=['label','area']
                                                       ,suffixes=('_particle', '_mask'))
    
    path = CSV_DIR  + 'mitosis/' + directory 
    #print("dri is", directory)
    #print(path)
    try:
        os.makedirs(path)
    except FileExistsError:
        #directory already exists
        pass
    check_nan_in_df = df.isnull().values.any()
    if check_nan_in_df >0:
        df=df.dropna()
        print("NaNs removed from")# %s " %( directory))
    
    df.to_csv(path + '/'  +  'particles'+'.csv',index=False) 

/var/folders/nw/b5trlz_d7819ystdc2gmcwbw0000gn/T/ipykernel_45771/1830156981.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_mask_final=pd.concat([df_mask_final,df_mask],ignore_index=True)
/var/folders/nw/b5trlz_d7819ystdc2gmcwbw0000gn/T/ipykernel_45771/1830156981.py:65: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_particle_final=pd.concat([df_particle_final,df_particle],ignore_index=True)


NaNs removed from


<Figure size 640x480 with 0 Axes>

In [ ]:
#particle intensity (average/mean) distribution and mask intensity (average/mean) distribution.
#T: mask/particle mean_intensity_mask/min_intensity_particle
#U: particle/mask mean_intesity_particle/mean_intensity_mask
#v Q/p in the excel sheet particle@kt:particle@cyto/mask@kt:mask@cyto

### Generate CSVs

In [7]:
# The paths are for the generated csv files (per experiment per condition)
interphase_csv=[]
mitosis_csv=[]
interphase_path = CSV_DIR + 'interphase'
mitosis_path=CSV_DIR + 'mitosis'

for folder in natsorted(glob.glob(interphase_path + '/*')):
    interphase_csv.append(interphase_path + '/' + folder.split('/')[-1].split('.')[0])
    
for folder in natsorted(glob.glob(mitosis_path + '/*')):
    mitosis_csv.append(mitosis_path + '/' + folder.split('/')[-1].split('.')[0])  

In [4]:
interphase_csv,mitosis_csv

(['/Users/Trupti/07-ImageAnalysis/Kinetochore-Particle-Tracker/AnalysisResults/SiwenMovie/KTProtein/interphase/Expt-1'],
 ['/Users/Trupti/07-ImageAnalysis/Kinetochore-Particle-Tracker/AnalysisResults/SiwenMovie/KTProtein/mitosis/Expt-1'])

In [22]:
# read interphase csvs to a dataframe
cell_count=1
path = GRAPHS_DIR + FOLDER
try:
    os.makedirs(path)
except FileExistsError:
    # directory already exists
    pass
df_graph_interphase=pd.DataFrame()
for i in range(len(interphase_csv)):
    df_interphase=pd.read_csv(interphase_csv[i]+'/'+'Particles.csv')
    
    df_interphase['mask@kt/mask@cyto']= df_interphase['mean_intensity_mask']/df_interphase['mask_cyto']
    df_interphase['particle@kt/particle@cyto']= df_interphase['mean_intensity_particle']/df_interphase['particle_cyto']
    df_interphase['mask@kt/mask@cyto:particle@kt/particle@cyto']= df_interphase['mask@kt/mask@cyto'] / df_interphase['particle@kt/particle@cyto']
    df_interphase['mean_intensity_mask/mean_intensity_particle']=df_interphase['mean_intensity_mask']/df_interphase['mean_intensity_particle']
    df_interphase['mean_intensity_particle/mean_intensity_mask']=df_interphase['mean_intensity_particle']/df_interphase['mean_intensity_mask']
    df_interphase['particle@kt/particle@cyto:mask@kt/mask@cyto']= df_interphase['particle@kt/particle@cyto']/df_interphase['mask@kt/mask@cyto']
    df_interphase['condition']='Interphase'
    df_interphase['cell']=FOLDER+"_"+'Interphase_'+str(cell_count)
    expt_name=re.sub(r'^.+/([^/]+)$', r'\1', interphase_csv[i])
    df_interphase['Experiment']= expt_name
    fname=FOLDER+'_interphase_'+ str(i)
    df_interphase_final=outliers_particle(df_interphase)
    df_interphase_final.to_csv(GRAPHS_DIR + FOLDER+'/' + fname +'.csv') 
    df_graph_interphase=pd.concat([df_graph_interphase,df_interphase_final],ignore_index=False)    
    cell_count+=1
#df_graph_interphase = df_graph_interphase.drop(columns=["Unnamed: 0"], axis=1)    

df_graph_interphase.to_csv(GRAPHS_DIR + FOLDER+'/' +FOLDER+ "_" +'interphase_graphs'+'.csv',index=False) 

In [23]:
# read mitosis csvs to a dataframe
cell_count=1
path = GRAPHS_DIR + FOLDER
try:
    os.makedirs(path)
except FileExistsError:
    # directory already exists
    pass
df_graph_mitosis=pd.DataFrame()
for i in range(len(mitosis_csv)):
    df_mitosis=pd.read_csv(mitosis_csv[i]+'/'+'Particles.csv')
    df_mitosis['mask@kt/mask@cyto']= df_mitosis['mean_intensity_mask']/df_mitosis['mask_cyto']
    df_mitosis['particle@kt/particle@cyto']= df_mitosis['mean_intensity_particle']/df_mitosis['particle_cyto']
    df_mitosis['mask@kt/mask@cyto:particle@kt/particle@cyto']= df_mitosis['mask@kt/mask@cyto'] / df_mitosis['particle@kt/particle@cyto']
    df_mitosis['mean_intensity_mask/mean_intensity_particle']=df_mitosis['mean_intensity_mask']/df_mitosis['mean_intensity_particle']
    df_mitosis['mean_intensity_particle/mean_intensity_mask']=df_mitosis['mean_intensity_particle']/df_mitosis['mean_intensity_mask']
    df_mitosis['particle@kt/particle@cyto:mask@kt/mask@cyto']= df_mitosis['particle@kt/particle@cyto']/df_mitosis['mask@kt/mask@cyto']
    df_mitosis['condition']='Mitosis'
    df_mitosis['cell']=FOLDER+"_"+'Mitosis_'+str(cell_count)
    expt_name=re.sub(r'^.+/([^/]+)$', r'\1', mitosis_csv[i])
    df_mitosis['Experiment']= expt_name
    fname=FOLDER+'_mitosis_'+ str(i)
    df_mitosis_final=outliers_particle(df_mitosis)
    df_mitosis_final.to_csv(GRAPHS_DIR + FOLDER+'/' + fname +'.csv') 
    df_graph_mitosis=pd.concat([df_graph_mitosis,df_mitosis_final],ignore_index=False)    
    cell_count+=1
#df_graph_mitosis = df_graph_mitosis.drop(columns=["Unnamed: 0"], axis=1)    

df_graph_mitosis.to_csv(GRAPHS_DIR + FOLDER+'/' +FOLDER+ "_" +'mitosis_graphs'+'.csv',index=False) 

In [24]:
df_graph=pd.DataFrame()
df_graph=pd.concat([df_graph,df_graph_interphase,df_graph_mitosis],ignore_index=True)
print("Number of particles for Interphase:  ",len(df_graph_interphase))
print("Number of particles for Mitosis:  ",len(df_graph_mitosis))
print(len(df_graph))
df_graph.head()
df_graph.to_csv(GRAPHS_DIR + FOLDER+'/' +FOLDER+ "_" + 'Siwen_combined_graphs'+'.csv',index=False) 
#df_graph.head()

Number of particles for Interphase:   154
Number of particles for Mitosis:   161
315


In [17]:
df_graph

,label,area,min_intensity_particle,max_intensity_particle,mean_intensity_particle,fname_particle,particle_cyto,particle_bg,min_intensity_mask,max_intensity_mask,...,particle@kt/particle@cyto,mask@kt/mask@cyto:particle@kt/particle@cyto,mean_intesity_mask/mean_intensity_particle,mean_intensity_particle/mean_intensity_mask,particle@kt/particle@cyto:mask@kt/mask@cyto,condition,cell,Experiment,mean_intesity_mask/mean_intesity_particle,mean_intesity_particle/mean_intesity_mask
0,0_1,5.0,507.0,653.0,597.400000,Expt2023_011_set5_RPE-1-p21-GFP-PCNA-mRuby_200...,631.403619,252.2050,242.0,681.0,...,0.946146,1.485515,0.761299,1.313544,0.673167,Interphase,KTProtein_Interphase_1,Expt-1,NaN,NaN
1,0_2,8.0,802.0,1146.0,1039.625000,Expt2023_011_set5_RPE-1-p21-GFP-PCNA-mRuby_200...,631.403619,252.2050,281.0,793.0,...,1.646530,0.902563,0.462547,2.161944,1.107956,Interphase,KTProtein_Interphase_1,Expt-1,NaN,NaN
2,0_3,8.0,54.0,426.0,214.625000,Expt2023_011_set5_RPE-1-p21-GFP-PCNA-mRuby_200...,631.403619,252.2050,188.0,739.0,...,0.339917,4.182147,2.143273,0.466576,0.239112,Interphase,KTProtein_Interphase_1,Expt-1,NaN,NaN
3,0_4,8.0,108.0,527.0,333.625000,Expt2023_011_set5_RPE-1-p21-GFP-PCNA-mRuby_200...,631.403619,252.2050,158.0,537.0,...,0.528386,2.095315,1.073810,0.931263,0.477255,Interphase,KTProtein_Interphase_1,Expt-1,NaN,NaN
4,0_5,5.0,511.0,928.0,701.600000,Expt2023_011_set5_RPE-1-p21-GFP-PCNA-mRuby_200...,631.403619,252.2050,320.0,598.0,...,1.111175,1.344432,0.688997,1.451386,0.743808,Interphase,KTProtein_Interphase_1,Expt-1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,18_1,8.0,1245.0,2803.0,1991.625000,Expt2023_011_set5_RPE-1-p21-GFP-PCNA-mRuby_200...,1323.699284,484.6325,291.0,961.0,...,1.504590,0.709909,NaN,NaN,1.408631,Mitosis,KTProtein_Mitosis_1,Expt-1,0.301575,3.315921
311,18_2,31.0,1481.0,4811.0,2762.967742,Expt2023_011_set5_RPE-1-p21-GFP-PCNA-mRuby_200...,1323.699284,484.6325,371.0,1579.0,...,2.087308,0.793334,NaN,NaN,1.260503,Mitosis,KTProtein_Mitosis_1,Expt-1,0.337015,2.967228
312,18_3,5.0,1118.0,2347.0,1731.800000,Expt2023_011_set5_RPE-1-p21-GFP-PCNA-mRuby_200...,1323.699284,484.6325,702.0,1162.0,...,1.308303,1.272287,NaN,NaN,0.785986,Mitosis,KTProtein_Mitosis_1,Expt-1,0.540478,1.850214
313,18_4,5.0,1279.0,2066.0,1629.200000,Expt2023_011_set5_RPE-1-p21-GFP-PCNA-mRuby_200...,1323.699284,484.6325,560.0,1285.0,...,1.230793,1.390555,NaN,NaN,0.719137,Mitosis,KTProtein_Mitosis_1,Expt-1,0.590719,1.692851
